In [1]:
import pandas as pd
import numpy as np
import csv
import json



# more readable exceptions
%pip install --quiet iwut
%load_ext iwut
%wut on
!pip install imblearn

Note: you may need to restart the kernel to use updated packages.
  Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
  Using cached imbalanced_learn-0.9.1-py3-none-any.whl (199 kB)


## Data Cleaning

In the survey, we collect features of voters (such as primary language and smoker or not)and their attitudes towards 3 propositions. We'll load our data and drop N/A values.

In [2]:
voting_data = pd.read_csv("data/voting-data.csv")
voting_data

,Timestamp,"California Proposition 1, Right to Reproductive Freedom Amendment (2022)","California Proposition 30, Tax on Income Above $2 Million for Zero-Emissions Vehicles and Wildfire Prevention Initiative (2022)","California Proposition 31, Flavored Tobacco Products Ban Referendum (2022)",What is your ethnicity?,My primary language is...,What is your average family income?,What is your gender?,What is your age?,What is your party self-identification?,...,How important are the following issues to you? [Healthcare],How important are the following issues to you? [Environment],How important are the following issues to you? [Education],How important are the following issues to you? [Women's Rights],Do you have healthcare?,Do you smoke?,I am registered to vote.,I plan to vote in the November 2022 election.,Unnamed: 20,Unnamed: 21
0,2022/07/20 11:16:12 PM MDT,Yes,Yes,Yes,Caucasian,English,"$100,000 - $400,000",Female,65+,Democrat,...,NaN,NaN,NaN,NaN,Yes,No,NaN,NaN,5.0,5.0
1,2022/07/20 11:36:03 PM MDT,Yes,Yes,Yes,Caucasian,English,"$100,000 - $400,000",Female,65+,Democrat,...,NaN,NaN,NaN,NaN,Yes,No,NaN,NaN,5.0,5.0
2,2022/07/21 9:42:16 AM MDT,Yes,Yes,Yes,Caucasian,English,"$100,000 - $400,000",Female,18-25,Democrat,...,Important,Very Important,Very Important,Very Important,Yes,No,No,Yes,NaN,NaN
3,2022/07/21 9:49:05 AM MDT,Yes,Yes,Yes,Caucasian,Spanish,"$50,000- $100,000",Female,65+,Republican,...,Very Important,Very Important,Very Important,Very Important,Yes,No,Yes,Yes,NaN,NaN
4,2022/07/21 12:19:15 PM MDT,No,Yes,Yes,Caucasian,English,"$100,000 - $400,000",Female,18-25,Democrat,...,Important,Important,Important,Important,Yes,No,Yes,Yes,NaN,NaN
5,2022/07/21 1:33:39 PM MDT,Yes,Yes,Yes,Caucasian,English,"$20,000 - $50,000",Female,65+,Democrat,...,Very Important,Very Important,Important,Very Important,Yes,No,Yes,Yes,NaN,NaN
6,2022/07/21 4:02:15 PM MDT,Yes,Yes,Yes,Asian,English,"$0 - $20,000",Male,18-25,Democrat,...,Very Important,Important,Important,Very Important,Yes,No,Yes,Yes,NaN,NaN
7,2022/07/21 4:12:39 PM MDT,Yes,Yes,No,Asian,English,"$100,000 - $400,000",Female,18-25,Democrat,...,Very Important,Very Important,Very Important,Very Important,Yes,No,Yes,Yes,NaN,NaN
8,2022/07/21 4:23:52 PM MDT,Yes,Yes,No,Asian,Chinese,"$100,000 - $400,000",Female,18-25,Independent,...,Somewhat Important,Somewhat Unimportant,Important,Somewhat Important,Yes,No,No,Maybe,NaN,NaN
9,2022/07/21 5:50:12 PM MDT,Yes,Yes,Yes,Caucasian,English,"$100,000 - $400,000",Male,65+,Democrat,...,Important,Important,Somewhat Important,Important,Yes,No,Yes,Yes,NaN,NaN


In [3]:
#Clean the data by dropping N/A Values
voting_data_dropped = voting_data.drop(columns = ["Timestamp", "Unnamed: 20", "Unnamed: 21"])

In [4]:
# Rename Propositions to make it easier to understand
voting_data_simplified = voting_data_dropped.rename(columns={"California Proposition 1, Right to Reproductive Freedom Amendment (2022)": "Prop-1", "California Proposition 30, Tax on Income Above $2 Million for Zero-Emissions Vehicles and Wildfire Prevention Initiative (2022)": "Prop-30", "California Proposition 31, Flavored Tobacco Products Ban Referendum (2022)":"Prop-31"})
voting_data_simplified

,Prop-1,Prop-30,Prop-31,What is your ethnicity?,My primary language is...,What is your average family income?,What is your gender?,What is your age?,What is your party self-identification?,What is your education background?,How important are the following issues to you? [Cost of Living],How important are the following issues to you? [Healthcare],How important are the following issues to you? [Environment],How important are the following issues to you? [Education],How important are the following issues to you? [Women's Rights],Do you have healthcare?,Do you smoke?,I am registered to vote.,I plan to vote in the November 2022 election.
0,Yes,Yes,Yes,Caucasian,English,"$100,000 - $400,000",Female,65+,Democrat,Bachelor's degree (for example: BA. BS),NaN,NaN,NaN,NaN,NaN,Yes,No,NaN,NaN
1,Yes,Yes,Yes,Caucasian,English,"$100,000 - $400,000",Female,65+,Democrat,"Master's degree (for example: MA, MS, MEng, ME...",NaN,NaN,NaN,NaN,NaN,Yes,No,NaN,NaN
2,Yes,Yes,Yes,Caucasian,English,"$100,000 - $400,000",Female,18-25,Democrat,High school,Important,Important,Very Important,Very Important,Very Important,Yes,No,No,Yes
3,Yes,Yes,Yes,Caucasian,Spanish,"$50,000- $100,000",Female,65+,Republican,Bachelor's degree (for example: BA. BS),Very Important,Very Important,Very Important,Very Important,Very Important,Yes,No,Yes,Yes
4,No,Yes,Yes,Caucasian,English,"$100,000 - $400,000",Female,18-25,Democrat,Some college,Important,Important,Important,Important,Important,Yes,No,Yes,Yes
5,Yes,Yes,Yes,Caucasian,English,"$20,000 - $50,000",Female,65+,Democrat,"Master's degree (for example: MA, MS, MEng, ME...",Very Important,Very Important,Very Important,Important,Very Important,Yes,No,Yes,Yes
6,Yes,Yes,Yes,Asian,English,"$0 - $20,000",Male,18-25,Democrat,Some college,Very Important,Very Important,Important,Important,Very Important,Yes,No,Yes,Yes
7,Yes,Yes,No,Asian,English,"$100,000 - $400,000",Female,18-25,Democrat,Bachelor's degree (for example: BA. BS),Very Important,Very Important,Very Important,Very Important,Very Important,Yes,No,Yes,Yes
8,Yes,Yes,No,Asian,Chinese,"$100,000 - $400,000",Female,18-25,Independent,Some college,Somewhat Important,Somewhat Important,Somewhat Unimportant,Important,Somewhat Important,Yes,No,No,Maybe
9,Yes,Yes,Yes,Caucasian,English,"$100,000 - $400,000",Male,65+,Democrat,Ph.D. or higher,Somewhat Unimportant,Important,Important,Somewhat Important,Important,Yes,No,Yes,Yes


## Prepare input and output for Machine Learning
We transform categorical data to numerical values, extract features as input and answers to propositions as output.


In [5]:
# This dataframe is used as input 
voting_data_input = voting_data_simplified.drop(columns = ["Prop-1", "Prop-30", "Prop-31"])
voting_data_input

,What is your ethnicity?,My primary language is...,What is your average family income?,What is your gender?,What is your age?,What is your party self-identification?,What is your education background?,How important are the following issues to you? [Cost of Living],How important are the following issues to you? [Healthcare],How important are the following issues to you? [Environment],How important are the following issues to you? [Education],How important are the following issues to you? [Women's Rights],Do you have healthcare?,Do you smoke?,I am registered to vote.,I plan to vote in the November 2022 election.
0,Caucasian,English,"$100,000 - $400,000",Female,65+,Democrat,Bachelor's degree (for example: BA. BS),NaN,NaN,NaN,NaN,NaN,Yes,No,NaN,NaN
1,Caucasian,English,"$100,000 - $400,000",Female,65+,Democrat,"Master's degree (for example: MA, MS, MEng, ME...",NaN,NaN,NaN,NaN,NaN,Yes,No,NaN,NaN
2,Caucasian,English,"$100,000 - $400,000",Female,18-25,Democrat,High school,Important,Important,Very Important,Very Important,Very Important,Yes,No,No,Yes
3,Caucasian,Spanish,"$50,000- $100,000",Female,65+,Republican,Bachelor's degree (for example: BA. BS),Very Important,Very Important,Very Important,Very Important,Very Important,Yes,No,Yes,Yes
4,Caucasian,English,"$100,000 - $400,000",Female,18-25,Democrat,Some college,Important,Important,Important,Important,Important,Yes,No,Yes,Yes
5,Caucasian,English,"$20,000 - $50,000",Female,65+,Democrat,"Master's degree (for example: MA, MS, MEng, ME...",Very Important,Very Important,Very Important,Important,Very Important,Yes,No,Yes,Yes
6,Asian,English,"$0 - $20,000",Male,18-25,Democrat,Some college,Very Important,Very Important,Important,Important,Very Important,Yes,No,Yes,Yes
7,Asian,English,"$100,000 - $400,000",Female,18-25,Democrat,Bachelor's degree (for example: BA. BS),Very Important,Very Important,Very Important,Very Important,Very Important,Yes,No,Yes,Yes
8,Asian,Chinese,"$100,000 - $400,000",Female,18-25,Independent,Some college,Somewhat Important,Somewhat Important,Somewhat Unimportant,Important,Somewhat Important,Yes,No,No,Maybe
9,Caucasian,English,"$100,000 - $400,000",Male,65+,Democrat,Ph.D. or higher,Somewhat Unimportant,Important,Important,Somewhat Important,Important,Yes,No,Yes,Yes


In [6]:
# Encoding all features into dummies variables
voting_data_input_dummies = pd.get_dummies(voting_data_input)
voting_data_input_dummies

,What is your ethnicity?_Asian,What is your ethnicity?_Black or African American,What is your ethnicity?_Caucasian,What is your ethnicity?_Hispanic or Latino(a),My primary language is..._Chinese,My primary language is..._English,My primary language is..._Spanish,"What is your average family income?_$0 - $20,000","What is your average family income?_$100,000 - $400,000","What is your average family income?_$20,000 - $50,000",...,How important are the following issues to you? [Women's Rights]_Somewhat Unimportant,How important are the following issues to you? [Women's Rights]_Very Important,Do you have healthcare?_No,Do you have healthcare?_Yes,Do you smoke?_No,Do you smoke?_Yes,I am registered to vote._No,I am registered to vote._Yes,I plan to vote in the November 2022 election._Maybe,I plan to vote in the November 2022 election._Yes
0,0,0,1,0,0,1,0,0,1,0,...,0,0,0,1,1,0,0,0,0,0
1,0,0,1,0,0,1,0,0,1,0,...,0,0,0,1,1,0,0,0,0,0
2,0,0,1,0,0,1,0,0,1,0,...,0,1,0,1,1,0,1,0,0,1
3,0,0,1,0,0,0,1,0,0,0,...,0,1,0,1,1,0,0,1,0,1
4,0,0,1,0,0,1,0,0,1,0,...,0,0,0,1,1,0,0,1,0,1
5,0,0,1,0,0,1,0,0,0,1,...,0,1,0,1,1,0,0,1,0,1
6,1,0,0,0,0,1,0,1,0,0,...,0,1,0,1,1,0,0,1,0,1
7,1,0,0,0,0,1,0,0,1,0,...,0,1,0,1,1,0,0,1,0,1
8,1,0,0,0,1,0,0,0,1,0,...,0,0,0,1,1,0,1,0,1,0
9,0,0,1,0,0,1,0,0,1,0,...,0,0,0,1,1,0,0,1,0,1


In [7]:
# Process output as numerical values 
map_values = {'Yes': 1, 'No': 0}
voting_data_simplified['Prop-1'] = voting_data_simplified['Prop-1'].map(map_values)
voting_data_simplified['Prop-30'] = voting_data_simplified['Prop-30'].map(map_values)
voting_data_simplified['Prop-31'] = voting_data_simplified['Prop-31'].map(map_values)
voting_data_simplified


,Prop-1,Prop-30,Prop-31,What is your ethnicity?,My primary language is...,What is your average family income?,What is your gender?,What is your age?,What is your party self-identification?,What is your education background?,How important are the following issues to you? [Cost of Living],How important are the following issues to you? [Healthcare],How important are the following issues to you? [Environment],How important are the following issues to you? [Education],How important are the following issues to you? [Women's Rights],Do you have healthcare?,Do you smoke?,I am registered to vote.,I plan to vote in the November 2022 election.
0,1,1,1,Caucasian,English,"$100,000 - $400,000",Female,65+,Democrat,Bachelor's degree (for example: BA. BS),NaN,NaN,NaN,NaN,NaN,Yes,No,NaN,NaN
1,1,1,1,Caucasian,English,"$100,000 - $400,000",Female,65+,Democrat,"Master's degree (for example: MA, MS, MEng, ME...",NaN,NaN,NaN,NaN,NaN,Yes,No,NaN,NaN
2,1,1,1,Caucasian,English,"$100,000 - $400,000",Female,18-25,Democrat,High school,Important,Important,Very Important,Very Important,Very Important,Yes,No,No,Yes
3,1,1,1,Caucasian,Spanish,"$50,000- $100,000",Female,65+,Republican,Bachelor's degree (for example: BA. BS),Very Important,Very Important,Very Important,Very Important,Very Important,Yes,No,Yes,Yes
4,0,1,1,Caucasian,English,"$100,000 - $400,000",Female,18-25,Democrat,Some college,Important,Important,Important,Important,Important,Yes,No,Yes,Yes
5,1,1,1,Caucasian,English,"$20,000 - $50,000",Female,65+,Democrat,"Master's degree (for example: MA, MS, MEng, ME...",Very Important,Very Important,Very Important,Important,Very Important,Yes,No,Yes,Yes
6,1,1,1,Asian,English,"$0 - $20,000",Male,18-25,Democrat,Some college,Very Important,Very Important,Important,Important,Very Important,Yes,No,Yes,Yes
7,1,1,0,Asian,English,"$100,000 - $400,000",Female,18-25,Democrat,Bachelor's degree (for example: BA. BS),Very Important,Very Important,Very Important,Very Important,Very Important,Yes,No,Yes,Yes
8,1,1,0,Asian,Chinese,"$100,000 - $400,000",Female,18-25,Independent,Some college,Somewhat Important,Somewhat Important,Somewhat Unimportant,Important,Somewhat Important,Yes,No,No,Maybe
9,1,1,1,Caucasian,English,"$100,000 - $400,000",Male,65+,Democrat,Ph.D. or higher,Somewhat Unimportant,Important,Important,Somewhat Important,Important,Yes,No,Yes,Yes


In [8]:
voting_output = voting_data_simplified[['Prop-1', 'Prop-30', 'Prop-31']]
voting_output 

,Prop-1,Prop-30,Prop-31
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,0,1,1
5,1,1,1
6,1,1,1
7,1,1,0
8,1,1,0
9,1,1,1


## Machine Learning
We split dataset into 80% training data and 20% test data. We want use some features of the voter to predict their attitudes on proposition 1, 30, and 31. We apply accuracy as measure because 'yes' and 'no' class are equally important to us, so we'd love to capture more true positive and true nagative.

In [9]:
# Import packages
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, confusion_matrix
from imblearn.ensemble import BalancedBaggingClassifier 

In [10]:
# Spliting the dataset into 80% training and 20% test data
X = voting_data_input_dummies
y1 = voting_output["Prop-1"]
y30 = voting_output["Prop-30"]
y31 = voting_output["Prop-31"]

X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y1, test_size=0.2, random_state=42)
X_train30, X_test30, y_train30, y_test30 = train_test_split(X, y30, test_size=0.2, random_state=42)
X_train31, X_test31, y_train31, y_test31 = train_test_split(X, y31, test_size=0.2, random_state=42)

## Explore: Use PCA to figure out dominant features
We're curious about whether any feature is a dominant one for predicting the result. We apply PCA on data for proposition 1.


Result: Even the most important feature could only explain 17% of the variance in data. Therefore, we need more than 1 feature to make predictions on answer to propositions. Thus we reject models like simple linear regression, and consider models like multiple linear regression.

In [11]:
pca = PCA(n_components=2)
pca = pca.fit(X)
comp = pca.components_

In [12]:
pca.explained_variance_ratio_

array([0.17373956, 0.1478949 ])

In [13]:
comp_x = comp[0]
comp_y = comp[1]
X_train1.columns[np.argmax(abs(comp_x))]

"How important are the following issues to you? [Women's Rights]_Important"

In [14]:
X_train1.columns[np.argmax(abs(comp_y))]

'What is your party self-identification?_Democrat'

## Model 1: Logistic Regression
We apply logistic regression as our goal is to predict a binary categorical value from as a linear function of features.

Result: Perfect performance on proposition 30, extremely poor at 31.

In [18]:
# Make classifications on Proposition 1
# Train the model
reg_1_classifier = LogisticRegression(random_state=42).fit(X_train1,y_train1)
# Check training accuracy
reg_train1_score = reg_1_classifier.score(X_train1,y_train1)
# Use the model to make predictions
reg_y_pred1 = reg_1_classifier.predict(X_test1)
# Check Prediction result
reg_test1_score = accuracy_score(reg_y_pred1,y_test1)

print("train score for Prop-1:"+ str(reg_train1_score)+", test score for Prop-1:" + str(reg_test1_score))

train score for Prop-1:0.9473684210526315, test score for Prop-1:0.8


In [19]:
# Make classifications on Proposition 30
reg_30_classifier = LogisticRegression(random_state=42).fit(X_train30,y_train30)
reg_train30_score = reg_30_classifier.score(X_train30,y_train30)
y_pred30 = reg_30_classifier.predict(X_test30)
reg_test30_score = accuracy_score(y_pred30,y_test30)
print("train score for Prop-30:"+ str(reg_train30_score)+", test score for Prop-30:" + str(reg_test30_score))

train score for Prop-30:1.0, test score for Prop-30:1.0


In [20]:
# Make classifications on Proposition 31
reg_31_classifier = LogisticRegression(random_state=42).fit(X_train31,y_train31)
reg_train31_score = reg_31_classifier.score(X_train31,y_train31)
y_pred31 = reg_31_classifier.predict(X_test31)
reg_test31_score = accuracy_score(y_pred31,y_test31)
print("train score for Prop-31:"+ str(reg_train31_score)+", test score for Prop-31:" + str(reg_test31_score))

train score for Prop-31:1.0, test score for Prop-31:0.4


## Model 2: Decision Tree
A decision tree is a tree of questions that must be answered in sequence to yield a predicted classification. The test score is fairly well.

In [51]:
voting_features = voting_data_simplified.iloc[:, 3:]

In [52]:
# Make classifications on Proposition 1
tree_1_classifier = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='not majority',
                                replacement=False,
                                random_state=42).fit(X_train1, y_train1)
tree_train1_score = tree_1_classifier.score(X_train1,y_train1)
tree_y_pred1 = tree_1_classifier.predict(X_test1)
tree_test1_score = accuracy_score(tree_y_pred1,y_test1)
print("train score for Prop-1:"+ str(tree_train1_score)+", test score for Prop-31:" + str(tree_test1_score))

train score for Prop-1:1.0, test score for Prop-31:0.6


In [53]:
# Make classifications on Proposition 30
tree_30_classifier = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='not majority',
                                replacement=False,
                                random_state=42).fit(X_train30, y_train30)
tree_train30_score = accuracy_score(tree_30_classifier.predict(X_train30),y_train30)
tree_y_pred30 = tree_30_classifier.predict(X_test30)
tree_test30_score = accuracy_score(tree_y_pred30,y_test30)
print("train score for Prop-30:"+ str(tree_train30_score)+", test score for Prop-30:" + str(tree_test30_score))

train score for Prop-30:1.0, test score for Prop-30:1.0


In [54]:
# Make classifications on Proposition 31
tree_31_classifier = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='not majority',
                                replacement=False,
                                random_state=42).fit(X_train31, y_train31)
tree_train31_score = accuracy_score(tree_31_classifier.predict(X_train31),y_train31)
tree_y_pred31 = tree_31_classifier.predict(X_test31)
tree_test31_score = accuracy_score(tree_y_pred31,y_test31)
print("train score for Prop-31:"+ str(tree_train31_score)+", test score for Prop-31:" + str(tree_test31_score))

train score for Prop-31:1.0, test score for Prop-31:0.6


## Model 3: Random Forests

Though decision trees are pretty good at predicting training datasets, overfitting problem exists. Therefore, random forest is used instead. We noticed that we have an unbalanced dataset, so we plan to oversample/undersample some classes. Since our dataset is small, we choose to oversample non-major classes. 


Result: For propositions 1 and 30, we have perfect test results. For Proposition 31, the test score is 0.4. Overall, it performs the best among all three models.

In [55]:
# Make classifications on Proposition 1
forest_1_classifier = BalancedBaggingClassifier(base_estimator = RandomForestClassifier(),
                                                sampling_strategy = 'not majority',
                                random_state=42).fit(X_train1, y_train1)
forest_train1_score = forest_1_classifier.score(X_train1,y_train1)
forest_y_pred1 = forest_1_classifier.predict(X_test1)
forest_test1_score = accuracy_score(forest_y_pred1,y_test1)
print("train score for Prop-1:"+ str(forest_train1_score)+", test score for Prop-1:" + str(forest_test1_score))

train score for Prop-1:1.0, test score for Prop-1:1.0


In [56]:
# Make classifications on Proposition 30
forest_30_classifier = BalancedBaggingClassifier(base_estimator = RandomForestClassifier(),
                                                 sampling_strategy = 'not majority',
                                random_state=42).fit(X_train30, y_train30)
forest_train30_score = forest_30_classifier.score(X_train30,y_train30)
forest_y_pred30 = forest_30_classifier.predict(X_test30)
forest_test30_score = accuracy_score(forest_y_pred30,y_test30)
print("train score for Prop-30:"+ str(forest_train30_score)+", test score for Prop-30:" + str(forest_test30_score))

train score for Prop-30:0.9473684210526315, test score for Prop-30:1.0


In [57]:
# Make classifications on Proposition 31
forest_31_classifier =  BalancedBaggingClassifier(base_estimator = RandomForestClassifier(),
                               sampling_strategy = 'not majority',
                                random_state=42).fit(X_train31, y_train31)
forest_train31_score = forest_31_classifier.score(X_train31,y_train31)
forest_y_pred31 = forest_31_classifier.predict(X_test31)
forest_test31_score = accuracy_score(forest_y_pred31,y_test31)
print("train score for Prop-31:"+ str(forest_train31_score)+", test score for Prop-31:" + str(forest_test31_score))

train score for Prop-31:1.0, test score for Prop-31:0.4


Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/popularity_contest/reporter.py", line 105, in report_popularity
    libraries = get_used_libraries(initial_modules, current_modules)
  File "/opt/conda/lib/python3.9/site-packages/popularity_contest/reporter.py", line 74, in get_used_libraries
    all_packages = get_all_packages()
  File "/opt/conda/lib/python3.9/site-packages/popularity_contest/reporter.py", line 51, in get_all_packages
    for f in dist.files:
TypeError: 'NoneType' object is not iterable



## We should proceeed with random forests.